# Start Interactive Dask Cluster

In [3]:
# this is for a strange bug with compute instances 
import os

os.system('sudo cp /etc/nginx/nginx.conf temp.conf') # stupid

nginx = ''

with open('temp.conf') as f:
    for line in f.readlines():
        if 'websocket/|/ws/' in line:
            nginx += line.replace('websocket/|/ws/', 'websocket/|/ws')
        else:
            nginx += line
       
with open('temp2.conf', 'w') as f:
    f.write(nginx)
    
os.system('sudo mv temp2.conf /etc/nginx/nginx.conf')
os.system('sudo service nginx restart')
os.system('rm temp.conf');

## Imports

Import all packages used in this notebook.

In [4]:
import os
import time
import socket

from IPython.core.display import HTML

from azureml.widgets import RunDetails
from azureml.train.estimator import Estimator
from azureml.core.runconfig import MpiConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace, Experiment, Datastore, Dataset, Environment

%matplotlib inline

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception cannot import name '_DistributedTraining'.


## Azure ML setup

Get the workspace.

In [5]:
ws = Workspace.from_config()
ws

Workspace.create(name='dask-cp-testing', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters-dasky-rg')

### Enter your name

Enter your name and virtual network information.

In [7]:
### name
name        = 'dask'             # REPLACE

### vnet settings
vnet_rg     = ws.resource_group  # replace if needed
vnet_name   = 'dasky-vnet'       # replace if needed
subnet_name = 'default'          # replace if needed

### azure ml names 
ct_name     = f'{name}-ct'
env_name    = f'{name}-env'

### trust but verify
verify = f'''
Name: {name}

vNET RG: {vnet_rg}
vNET name: {vnet_name}
vNET subnet name: {subnet_name}

Compute target: {ct_name}
Environment name: {env_name}
'''

print(verify)


Name: dask

vNET RG: copeters-dasky-rg
vNET name: dasky-vnet
vNET subnet name: default

Compute target: dask-ct
Environment name: dask-env



### Create VM pool

Create Azure ML VM pool for creating remote dask cluster(s).

In [8]:
if ct_name not in ws.compute_targets:
    # create config for Azure ML cluster
    # change properties as needed
    config = AmlCompute.provisioning_configuration(
             vm_size                       = 'STANDARD_DS13_V2', # 8 core 56 GiB 112 SSD 
             min_nodes                     = 0,
             max_nodes                     = 100,
             vnet_resourcegroup_name       = vnet_rg,              
             vnet_name                     = vnet_name,            
             subnet_name                   = subnet_name,          
             idle_seconds_before_scaledown = 300
    )
    ct = ComputeTarget.create(ws, ct_name, config)
    ct.wait_for_completion(show_output=True)    
else:
    ct = ws.compute_targets[ct_name]
    
ct

Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


AmlCompute(workspace=Workspace.create(name='dask-cp-testing', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters-dasky-rg'), name=dask-ct, id=/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/copeters-dasky-rg/providers/Microsoft.MachineLearningServices/workspaces/dask-cp-testing/computes/dask-ct, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

## Startup cluster

Start the run now. The first time, this will take 

### Mount Compute Instance code fileshare

This will create the compute instance code fileshare as a datastore. The default name `code-391ff5ac-6576-460f-ba4d-7e03433c68b6` and has the same credentials as the default fileshare for the workspace. This will be mounted for easy access to notebooks on the cluster.

In [9]:
codefileshare = 'codefileshare'

if codefileshare not in ws.datastores:
    Datastore.register_azure_file_share(ws, codefileshare,
                                        'code-391ff5ac-6576-460f-ba4d-7e03433c68b6',                    
                                        account_name = ws.datastores['workspacefilestore'].account_name, 
                                        account_key  = ws.datastores['workspacefilestore'].account_key   
                                       )

### Get data

This will get NOAA ISD Weather data which is used in the demo. If you already have data, you can skip this.

In [10]:
daskdata = 'noaa-isd-files'

if daskdata not in ws.datasets:
    ds = Dataset.File.from_files('https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather/*/*/*.parquet', validate=False)
    os.system('sudo chmod 777 /mnt')
    ds.download('/mnt/data/isd')
    ws.datastores['gen2'].upload('/mnt/data/isd', '/noaa-isd')
    ds = ds.register(ws, daskdata)

DatasetExecutionError: The operation has been canceled.

### Start cluster

This will soon be replaced.

In [11]:
from dask_cloudprovider import AzureMLCluster

In [13]:
packages = ['mpi4py',
            'distributed',
            'dask[complete]',
            'dask-ml[complete]',
            'fastparquet',
            'pyarrow',
            'jupyterlab',
            'joblib',
            'notebook',
            'adlfs', 
            'fsspec', 
            'azureml-sdk[notebooks]',
            'azureml-dataprep[fuse]',
            'lz4']

env = Environment(name=env_name)

for package in packages:
    env.python.conda_dependencies.add_pip_package(package)

In [19]:
cluster = AzureMLCluster(ws, ct, env, jupyter=True, asynchronous=False, datastores=[ws.datastores[datastore] for datastore in ws.datastores])

dask-cloudprovider
############################## Setting up cluster ##############################
########################## Submitting the experiment ###########################
####################### Waiting for scheduler node's IP ########################
........................................................................................................................................................................................................


########################## Scheduler: 10.11.0.12:8786 ##########################
############################# Scaling to 1 workers #############################
############################### Scaling is done ################################


In [20]:
cluster.run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-cloudprovider,dask-cloudprovider_1581220435_df00681e,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [35]:
cluster

In [22]:
cluster.connect_cluster()


Setting up port forwarding...
Cluster is ready to use.


In [24]:
cluster.jupyter_link

'https://cody-9000.westeurope.instances.azureml.net/lab?token=43f429344af211eabfe5000d3aa82394'

In [25]:
cluster.dashboard_link

'https://cody-9001.westeurope.instances.azureml.net/status'

In [26]:
cluster.scale(10)

In [34]:
nodes = len(cluster.workers_list)+1
nodes

10

In [36]:
cluster.close()

################### Scheduler and workers are disconnected. ####################


In [12]:
help(AzureMLCluster)

Help on class AzureMLCluster in module dask_cloudprovider.providers.azure.azureml:

class AzureMLCluster(distributed.deploy.cluster.Cluster)
 |  Deploy a Dask cluster using Azure ML
 |  
 |  This creates a dask scheduler and workers on an Azure ML Compute Target. 
 |  
 |  Parameters
 |  ----------
 |  workspace: azureml.core.Workspace (required)
 |      Azure ML Workspace - see https://aka.ms/azureml/workspace
 |  
 |  compute_target: azureml.core.ComputeTarget (required)
 |      Azure ML Compute Target - see https://aka.ms/azureml/computetarget
 |  
 |  environment_definition: azureml.core.Environment (required)
 |      Azure ML Environment - see https://aka.ms/azureml/environments
 |  
 |  experiment_name: str (optional)
 |      The name of the Azure ML Experiment used to control the cluster. 
 |  
 |      Defaults to ``dask-cloudprovider``. 
 |  
 |  initial_node_count: int (optional)
 |      The initial number of nodes for the Dask Cluster.
 |  
 |      Defaults to ``1``. 
 |  
 |